# Vertex AI 搜尋，包含篩選器和元資料

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/doggy8088/generative-ai/blob/main/search/search_filters_metadata.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中執行
    </a>
  </td>
  <td>
    <a href="https://github.com/doggy8088/generative-ai/blob/main/search/search_filters_metadata.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上檢視
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/blob/main/search/search_filters_metadata.zh.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中開啟
    </a>
  </td>
</table>


---

* 作者： [Ruchika Kharwar](https://github.com/rasalt)
* 建立時間： 2023 年 9 月 5 日

---

## 目標

本筆記本示範如何在 [Vertex AI Search](https://cloud.google.com/generative-ai-app-builder/docs/introduction) 的搜尋要求中使用 [篩選器和元資料](https://cloud.google.com/generative-ai-app-builder/docs/filter-search-metadata)。

這適用於包含元資料的非結構化應用程式。你可以使用元資料欄位將搜尋範圍限制在特定文件集合。


在筆記本中使用的服務：

- ✅ Vertex AI Search for document search and retrieval


## 安裝先備條件

如果在 Colab 中執行，請將先備條件安裝到執行階段。否則假設筆記本電腦在 Vertex AI Workbench 中執行。在這種情況下，建議使用 `--user` 選項，透過終端機安裝先備條件。


In [2]:
%pip install -q google-cloud-discoveryengine==0.11.2 --upgrade --user


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### 重新啟動目前的執行階段

要在此 Jupyter 執行階段中使用新安裝的套件，你必須重新啟動執行階段。你可以執行下列Cell來執行此項操作，如此將重新啟動目前的Kernel。


In [2]:
# Restart kernel after installs so that your environment can access the new packages
import IPython
import time

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ Kernel將重新啟動。請等待它完成，再繼續執行下一個步驟。⚠️</b>
</div>


## 驗證

如果在 Google Colab 中執行，請使用 `google.colab.google.auth` 驗證，否則假設在 Vertex AI Workbench 中執行。


In [4]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

from google.auth import default

creds, _ = default()

## 設定筆記本環境


## 數據儲存庫的元資料

資料儲存庫 `<DATA_STORE_ID>` 的元資料如下所示：

```json
{
  "id": "1",
  "structData": {
    "title": "Document1",
    "category": [
      "PersonaA"
    ],
    "name": "Document1"
  },
  "content": {
    "mimeType": "application/pdf",
    "uri": "gs://<BUCKETNAME>/data/Document1"
  }
}
```

```json
{
  "id": "2",
  "structData": {
    "title": "Document2",
    "category": [
      "PersonaA",
      "PersonaB"
    ],
    "name": "Document2"
  },
  "content": {
    "mimeType": "application/pdf",
    "uri": "gs://<BUCKETNAME>/data/Document2"
  }
}
```


### 設定以下常數，以反映你的環境


In [ ]:
PROJECT_ID = "<PROJECT_ID>"
LOCATION = "global"  # Replace with your data store location
DATA_STORE_ID = "<DATA_STORE_ID>"

### REST API 範例


過濾器 `name: ANY("Document1")` 可確保查詢僅針對 `name` 符合 `Document1` 的文件。


In [ ]:
%%bash -s "$PROJECT_ID" "$LOCATION" "$DATA_STORE_ID"

project_id=$1
location=$2
data_store_id=$3

curl -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
"https://discoveryengine.googleapis.com/v1beta/projects/$project_id/locations/$location/collections/default_collection/dataStores/$data_store_id/servingConfigs/default_search:search" \
-d '{
"query": "claim",
"filter": "name: ANY(\"Document1\")"
}'


篩選器 `類別: ANY("角色乙")` 可確保查詢僅針對 `name` 與 `Document1` 配對的文件執行。


In [ ]:
%%bash -s "$PROJECT_ID" "$LOCATION" "$DATA_STORE_ID"

project_id=$1
location=$2
data_store_id=$3

curl -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
"https://discoveryengine.googleapis.com/v1beta/projects/$project_id/locations/$location/collections/default_collection/dataStores/$data_store_id/servingConfigs/default_search:search" \
-d '{
"query": "claims",
"filter": "category: ANY(\"PersonaB\")"
}'


### Python 程式碼等同


In [ ]:
from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine_v1beta as discoveryengine


def search_data_store(
    project_id: str,
    location: str,
    data_store_id: str,
    search_query: str,
    filter_str: str,
) -> discoveryengine.SearchResponse:
    #  For more information, refer to:
    # https://cloud.google.com/generative-ai-app-builder/docs/locations#specify_a_multi-region_for_your_data_store
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )

    # Create a client
    client = discoveryengine.SearchServiceClient(client_options=client_options)

    # The full resource name of the search engine serving config
    # e.g. projects/{project_id}/locations/{location}/dataStores/{data_store_id}/servingConfigs/{serving_config_id}
    serving_config = client.serving_config_path(
        project=project_id,
        location=location,
        data_store=data_store_id,
        serving_config="default_config",
    )

    # Optional: Configuration options for search
    # Refer to the `ContentSearchSpec` reference for all supported fields:
    # https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest.ContentSearchSpec
    content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
        # For information about snippets, refer to:
        # https://cloud.google.com/generative-ai-app-builder/docs/snippets
        snippet_spec=discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
            return_snippet=True
        ),
        extractive_content_spec=discoveryengine.SearchRequest.ContentSearchSpec.ExtractiveContentSpec(
            max_extractive_answer_count=5,
            max_extractive_segment_count=1,
        ),
        # For information about search summaries, refer to:
        # https://cloud.google.com/generative-ai-app-builder/docs/get-search-summaries
        summary_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
            summary_result_count=5,
            include_citations=True,
            ignore_adversarial_query=False,
            ignore_non_summary_seeking_query=False,
        ),
    )

    # Refer to the `SearchRequest` reference for all supported fields:
    # https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest
    request = discoveryengine.SearchRequest(
        serving_config=serving_config,
        query=search_query,
        filter=filter_str,
        page_size=5,
        content_search_spec=content_search_spec,
        query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
            condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
        ),
        spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        ),
    )

    response = client.search(request)
    return response

In [ ]:
search_query = "how to process a claim"
filter_str = 'name: ANY("Document1")'

results = search_data_store(
    PROJECT_ID, LOCATION, DATA_STORE_ID, search_query, filter_str
)

print(f"\nQuestion: '{search_query}'\n\n")
print("Summary" + "-" * 40)
print(results.summary.summary_text)

print("Raw Results" + "-" * 40)
print(results)

下面是一個稍為複雜，由 2 個 metadata 值而成的篩選器


In [ ]:
search_query = "how to process a claim"
filter_str = 'name: ANY("Document1") AND category: ANY("PersonaA")'

results = search_data_store(
    PROJECT_ID, LOCATION, DATA_STORE_ID, search_query, filter_str
)

print(f"\nQuestion: '{search_query}'\n\n")
print("Summary" + "-" * 40)
print(results.summary.summary_text)

print("Raw Results" + "-" * 40)
print(results)